In [1]:
def fullVisit(isTest):
    visits = dict()
    n = 0
    with open(('test' if isTest==1 else 'train')+'.csv','r') as f:
        for line in f:
            if n == 0:
                n += 1
                pass
            else:
                vals = line.strip().split(',')
                typ = -1 if isTest==1 else int(vals[0])
                visit = int(vals[1-isTest])
                count = int(vals[4-isTest])
                
                val = visits.get(visit,[typ,vals[2-isTest],
                                        dict(),dict(),dict(),
                                        dict(),dict(),dict()])
                x = vals[5-isTest]
                val[2][x] = val[2].get(x,0) + count
                if count < 0:
                    val[5][x] = 1#val[5].get(x,0) - count
                
                x = vals[5-isTest]+'.'+vals[6-isTest]
                val[3][x] = val[3].get(x,0) + count
                if count < 0:
                    val[6][x] = 1#val[6].get(x,0) - count
                    
                x = vals[5-isTest]+'.'+vals[3-isTest]
                val[4][x] = val[4].get(x,0) + count
                if count < 0:
                    val[7][x] = 1#val[7].get(x,0) - count
                
                visits[visit] = val
    return visits
visits = fullVisit(0)

In [2]:
def getS(val,catXs):
    for cat in val:
        catXs[cat] = catXs.get(cat,0) + 1
        
wks, typs, cat1s, cat2s, cat3s, cat4s, cat5s, cat6s =\
dict(), dict(), dict(), dict(), dict(), dict(), dict(), dict()

for visit,val in visits.items():
    typ = val[0]
    typs[typ] = typs.get(typ,0) + 1
    wk = val[1]
    wks[wk] = wks.get(wk,0) + 1
    getS(val[2],cat1s)
    getS(val[3],cat2s)
    getS(val[4],cat3s)
    getS(val[5],cat4s)
    getS(val[6],cat5s)
    getS(val[7],cat6s)

In [3]:
def toItems(vals,rev=False):
    out, i = dict(), 0
    for val in sorted(vals):
        if rev:
            out[val] = i
        else:
            out[i] = val
        i += 1
    return out

def dicRev(dic):
    dicto = dict()
    for x,y in dic.items():
        dicto[y] = x
    return dicto

typs_ = toItems(typs,True)
wks_ = toItems(wks)
cat1s_ = toItems(cat1s)
cat2s_ = toItems(cat2s)
cat3s_ = toItems(cat3s)
cat4s_ = toItems(cat4s)
cat5s_ = toItems(cat5s)
cat6s_ = toItems(cat6s)

typs_x = dicRev(typs_)
wks_x = dicRev(wks_)
cat1s_x = dicRev(cat1s_)
cat2s_x = dicRev(cat2s_)
cat3s_x = dicRev(cat3s_)
cat4s_x = dicRev(cat4s_)
cat5s_x = dicRev(cat5s_)
cat6s_x = dicRev(cat6s_)

In [4]:
import numpy as np
from scipy.sparse import dok_matrix

def doTrainX(visits,tbl):
    
    def doRecs(vals,catXs,i_plus):
        for x,y in vals.items():
            i = catXs.get(x,-1)
            if i != -1:
                recs[n,i+i_plus] = y
                #print(i+i_plus,y)
                
    lw, lc1, lc2, lc3, lt =\
    len(wks), len(cat1s), len(cat2s), len(cat3s), len(typs)
    lc4, lc5, lc6 = len(cat4s), len(cat5s), len(cat6s)
    
    lall = lw+lc1+lc2+lc3+lc4+lc5+lc6
    tps, n = [], 0
    recs = dok_matrix((len(visits),lall), dtype=np.int16)
    for visit,vals in visits.items():
        if tbl == 'train':
            tps.append(typs_[vals[0]])
        else:
            tps.append(visit)
        
        i = wks_x.get(vals[1],-1)
        if i != -1:
            recs[n,i] = 1
        
        doRecs(vals[2],cat1s_x,lw)
        doRecs(vals[3],cat2s_x,lw+lc1)
        doRecs(vals[4],cat3s_x,lw+lc1+lc2)
        doRecs(vals[5],cat4s_x,lw+lc1+lc2+lc3)
        doRecs(vals[6],cat5s_x,lw+lc1+lc2+lc3+lc4)
        doRecs(vals[7],cat6s_x,lw+lc1+lc2+lc3+lc4+lc5)
        
        """
        out_ = [(1 if wks_[i] == vals[1] else 0) for i in range(lw)] +\
              [vals[2].get(cat1s_[i],0) for i in range(lc1)] +\
              [vals[3].get(cat2s_[i],0) for i in range(lc2)] +\
              [vals[4].get(cat3s_[i],0) for i in range(lc3)]
            
        for i in range(lall):
            if out_[i] != 0:
                recs[n,i] = out_[i]
        """        
        #break
        n += 1
        if n % 1000 == 0:
            print(n)
    return tps, recs

In [5]:
labels, train = doTrainX(visits,'train')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000


In [6]:
ids, test = doTrainX(fullVisit(1),'test')

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000


In [7]:
numCat = len(set(labels))
numCat

38

In [12]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import roc_curve, auc, f1_score, roc_auc_score, accuracy_score
from sklearn.cross_validation import KFold, train_test_split

def xgboost_pred(train,labels,test,folds=0,scale=0.6):
    def doTrain(labels):
        #train using early stopping and predict
        watchlist = [(xgtrain, 'train'),(xgval, 'val')]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=stopped)#, feval=evalerror)
        predsX = (model.predict(xgtest,ntree_limit=model.best_iteration),
                  model.best_score, model)
        print('best_score',predsX[1])
        preds.append(predsX)
        
    params = {}
    #params["booster"] = "gblinear"#gbtree"
    params["objective"] = "multi:softprob"#"multi:softmax"
    params["num_class"] = numCat
    params["eta"] = 0.005
    stopped = 300
    params["min_child_weight"] = 1
    params["subsample"] = 0.7
    params["colsample_bytree"] = 0.5
    
    params["scale_pos_weight"] = 1#scale#0.6
    
    params["max_depth"] = 6
    params["n_estimators"] = 4000
    
    #params["gamma"] = 0.8
    params["silent"] = 1
    params["eval_metric"] = "mlogloss"
    
    plst = list(params.items())

    #Using 5000 rows for early stopping. 
    offset = 2000
    num_rounds = 100000
    xgtest = xgb.DMatrix(test)
    
    preds = []
    if folds > 1:
        kf = KFold(labels.shape[0], n_folds=folds)#, shuffle=True, random_state=rng)
        for train_index, test_index in kf:
            xgtrain = xgb.DMatrix(train[train_index], label=labels[train_index])
            xgval = xgb.DMatrix(train[test_index], label=labels[test_index])
            doTrain(labels[test_index])
    else:
        #create a train and validation dmatrices
        if folds in [0,1]:
            xgtrain = xgb.DMatrix(train[offset:,:], label=labels[offset:])
            xgval = xgb.DMatrix(train[:offset,:], label=labels[:offset])
            doTrain(labels[:offset])
        else:
            folds = -folds
            df = 0
            f1,f2 = offset*folds+df, offset*(folds+1)+df
            train1 = np.concatenate((train[0:f1,:],train[f2:,:]),axis=0)
            labels1 = np.concatenate((labels[0:f1],labels[f2:]),axis=0)
            train2 = train[f1:f2,:]
            labels2 = labels[f1:f2]
            print(len(train1),len(labels1),len(train2),len(labels2))
            xgtrain = xgb.DMatrix(train1, label=labels1)
            xgval = xgb.DMatrix(train2, label=labels2)
            doTrain(labels2)            
    return preds

preds = xgboost_pred(train,labels,test,0)

Will train until val error hasn't decreased in 300 rounds.
[0]	train-mlogloss:3.612358	val-mlogloss:3.609150
[1]	train-mlogloss:3.591228	val-mlogloss:3.585234
[2]	train-mlogloss:3.569026	val-mlogloss:3.560868
[3]	train-mlogloss:3.550674	val-mlogloss:3.541225
[4]	train-mlogloss:3.532196	val-mlogloss:3.521851
[5]	train-mlogloss:3.512714	val-mlogloss:3.500284
[6]	train-mlogloss:3.493237	val-mlogloss:3.478839
[7]	train-mlogloss:3.473469	val-mlogloss:3.456953
[8]	train-mlogloss:3.455788	val-mlogloss:3.437872
[9]	train-mlogloss:3.437615	val-mlogloss:3.418355
[10]	train-mlogloss:3.419497	val-mlogloss:3.398893
[11]	train-mlogloss:3.402273	val-mlogloss:3.381063
[12]	train-mlogloss:3.386008	val-mlogloss:3.363188
[13]	train-mlogloss:3.369320	val-mlogloss:3.345279
[14]	train-mlogloss:3.354573	val-mlogloss:3.329820
[15]	train-mlogloss:3.338449	val-mlogloss:3.312705
[16]	train-mlogloss:3.322721	val-mlogloss:3.295634
[17]	train-mlogloss:3.309361	val-mlogloss:3.281959
[18]	train-mlogloss:3.294154	val-

best_score 0.63852


Stopping. Best iteration:
[28224]	train-mlogloss:0.245395	val-mlogloss:0.638520



In [16]:
%matplotlib inline
#xgb.plot_importance(preds[0][2])
len(sorted(preds[0][2].get_fscore().items(),key=lambda f: -f[1]))
#6518

18400

In [14]:
import pickle
with open('importance_full2.pk','wb') as fo:
    pickle.dump(sorted(preds[0][2].get_fscore().items(),key=lambda f: -f[1]), fo)

In [15]:
z = preds[0][0]
with open('result26.csv','w') as f:
    f.write(','.join(['"VisitNumber"']+['"TripType_'+str(x)+'"' for x,y in sorted(typs_.items())])+'\n')
    for i in range(len(ids)):
        f.write(','.join([str(ids[i])]+[str(j) for j in z[i]])+'\n')

In [10]:
for i in range(19):
    """
    preds_ = preds[i][0].astype(int)
    preds_ = pd.DataFrame({"ID": ids, "y": preds_})
    preds_ = preds_.set_index('ID')
    preds_.to_csv('result'+str(i)+'.csv')
    """
    print(i,preds[i][1])

0 0.788202
1 0.851402
2 0.79998
3 0.791688
4 0.815227
5 0.893484
6 0.835963
7 0.821291
8 0.820476
9 0.873758
10 0.923672
11 0.902607
12 0.875927
13 0.833284
14 0.840803
15 0.823168
16 0.764165
17 0.844184
18 0.837569


0 0.788202
1 0.851402
2 0.79998
3 0.791688
4 0.815227
5 0.893484
6 0.835963
7 0.821291
8 0.820476
9 0.873758
10 0.923672
11 0.902607
12 0.875927
13 0.833284
14 0.840803
15 0.823168
16 0.764165
17 0.844184
18 0.837569

In [88]:
for x in visits:
    y = visits[x]
    if len(y[0]) > 1 or len(y[1]) > 1:
        print(x,y)

In [77]:
import pandas as pd

train = pd.read_csv('train.csv')
def fillnaDF(train):
    for col in train:
        dt = train[col].dtype
        if dt == int or dt == float:
            train[col] = train[col].fillna(0 if col == 'ScanCount' else -999)
            if col in ['Upc','FinelineNumber']:
                train[col] = train[col].astype(int)
                if col == 'Upc':
                    train[col] = train.apply(lambda f: -998 
                                             if f.Upc == -999 and f.DepartmentDescription == 'PHARMACY RX' 
                                             else f.Upc,axis=1)
        elif dt == object:
            train[col] = train[col].fillna('-999')
fillnaDF(train)

train[(train[col] == -998) & (train.DepartmentDescription == 'PHARMACY RX')][col]

In [67]:
test.head(1)

,VisitNumber,Weekday,Upc,ScanCount,DepartmentDescription,FinelineNumber
0,1,Friday,72503389714,1,SHOES,3002


In [80]:
train.groupby(['Upc','DepartmentDescription']).count()
#train.Upc = train.Upc.astype(int)
#train[train.Upc>=9.781633e+11]

,,TripType,VisitNumber,Weekday,ScanCount,FinelineNumber
Upc,DepartmentDescription,,,,,
-999,-999,1361,1361,1361,1361,1361
-998,PHARMACY RX,2768,2768,2768,2768,2768
834,BAKERY,7,7,7,7,7
3032,PRODUCE,2,2,2,2,2
3035,PRODUCE,61,61,61,61,61
3066,PRODUCE,1,1,1,1,1
3082,PRODUCE,405,405,405,405,405
3100,PRODUCE,10,10,10,10,10
3107,PRODUCE,396,396,396,396,396
